## Registering the LGG NIFTIS

For each exam, we want to register everything to the following (in order): 
1. T1 pre-contrast Ax
2. T1 post-contrast Ax
3. T2 FSE Ax 
4. T1 (other orientation) post-contrast 
5. T2 FSE (other orientation)
6. T2 FLAIR Ax
7. T2 FLAIR (other orientation)

We want to use the following command: 

` BRAINSFit --fixedVolume $fixed_vol.nii.gz --movingVolume $moving_vol.nii.gz --outputVolume $output_vol.nii.gz --outputTransform $transform_output.tfm  --transformType Rigid`

But first we need to figure out which are the fixed volumes; this will require labeling. 

Let's use code derived from `http://localhost:8890/notebooks/data/svcf/labels/TCGA-GBM/GenerateGroundTruthUsingDCMdump.ipynb` 

This function will take in a patient file and output whether the T1 images have contrast: 

In [4]:
import glob
import subprocess as sub 
from subprocess import * 
import os 
from pathlib import Path
import pandas as pd

In [5]:
def find_if_t1_image_has_contrast(outfile, root_dir): 
    patient_images = glob.glob(str(root_dir)+ '/TCGA-LGG/'+outfile.split('.')[0]+"/*/*")
    
    image_contrast_map = {}
    for j in patient_images: 
        os.chdir(j)
#         print(j)
        command = "dcmdump 000000.dcm | grep -i '(0018,0010)' "
        result = sub.Popen(command, stdout=sub.PIPE, stderr = sub.PIPE, shell = True)
        out, err = result.communicate()
        out = out.decode('utf-8')
        if out: 
            image_contrast_map[j.split('/')[-1].split('.')[0]] = 1
        else: 
            image_contrast_map[j.split('/')[-1].split('.')[0]] = 0
            
    return image_contrast_map
            

In [6]:
root_dir = Path("/working/lupolab/julia/tcia_analysis/datasets/")

In [54]:
image_contrast_map = find_if_t1_image_has_contrast('TCGA-CS-5396', root_dir)

In [55]:
image_contrast_map

{'302-3D_MOTSA_sense-85827': 0,
 '403-eBo-48146': 0,
 '901-FFE_FOR_BLEED-67362': 0,
 '402-dADC_MAP-53538': 0,
 '1201-T1_COR_SE-91770': 0,
 '701-T2WTSE-70754': 0,
 '501-T1W__SE-58849': 0,
 '305-RT__COW-39894': 0,
 '404-eB1000i-30080': 0,
 '1301-T1_SAG_SE-45175': 0,
 '1101-T1_AX_SE-54608': 0,
 '801-PDWTSE-64364': 0,
 '304-LT_COW-25699': 0,
 '306-POSTERIOR-47371': 0,
 '301-3D_MOTSA_sense-80049': 0,
 '1001-T1_AX__SE-75237': 0,
 '601-T2_AX_FLAIR-98890': 0,
 '303-COW-38870': 0}

In [53]:
outfile = 'TCGA-CS-5396'

In [10]:
# for image in patient_images: 
#     series_description = image.split('/')[-1].split('.')[0]
#     print(series_description)

In [11]:
patient_images = glob.glob(str(root_dir)+ '/TCGA-LGG-nifti/'+outfile.split('.')[0]+"/*/*")

In [12]:
patient_images

['/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5872/12-03-1995-MRI_BRAIN_WWO_CONTRAST-92374/2-SAG_T1-87966.nii.gz',
 '/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5872/12-03-1995-MRI_BRAIN_WWO_CONTRAST-92374/1-3_PLANE_LOC-42073.nii.gz',
 '/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5872/12-03-1995-MRI_BRAIN_WWO_CONTRAST-92374/5-AXIAL_DTI-93315.nii.gz',
 '/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5872/12-03-1995-MRI_BRAIN_WWO_CONTRAST-92374/500-Average_DC-62164.nii.gz',
 '/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5872/12-03-1995-MRI_BRAIN_WWO_CONTRAST-92374/501-Fractional_Ansio.-46385.nii.gz',
 '/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5872/12-03-1995-MRI_BRAIN_WWO_CONTRAST-92374/7-AXIAL_PERFUSION-64124.nii.gz',
 '/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5872/12-03-1995-MRI_BRAIN_WWO_CONTRAST-92374/9-COR_T1

In [13]:
series_description = patient_images[0].split('/')[-1].split('.')[0]

In [37]:
def create_single_label_dataframe(outfile, image_contrast_map): 
    
    patient_images = glob.glob(str(root_dir)+ '/TCGA-LGG-nifti/'+outfile.split('.')[0]+"/*/*")

    #instantiate new label: 
    newlabel_df = pd.DataFrame()
    
    #cycle thru items in label, 
    for image in patient_images: 
        
        series_description = image.split('/')[-1].split('.')[0]
        
        if 't1' in series_description.lower() or 'spgr' in series_description.lower(): 
            if image_contrast_map[series_description] == 1: 
                new_gt = 'T1C'
                
            elif 'post' in series_description.lower(): 
                new_gt = 'T1C'
                
            else: 
                new_gt = "T1"
        elif 'flair' in series_description.lower() and 't1' not in series_description.lower(): 
            new_gt = 'T2_FLAIR'
        elif ('t2' in series_description.lower() or 'fse' in series_description.lower() )and 'flair' not in series_description.lower() and 'dw' not in series_description.lower(): 
            new_gt = 'T2'
            
        else: 
            new_gt = 'OTHER'
            
        newlabel_df_line = {'image': image, 'sd': series_description, 'ground_truth': new_gt}
#         print(newlabel_df_line)
        newlabel_df = newlabel_df.append(newlabel_df_line, ignore_index = True)
    
    return newlabel_df

In [38]:
df_example = create_single_label_dataframe(outfile, image_contrast_map)

In [39]:
outfiles = os.listdir('/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti')

In [40]:
full_label_df = pd.DataFrame()
for outfile in outfiles: 
    print(outfile)
    image_contrast_map = find_if_t1_image_has_contrast(outfile, root_dir)
    single_label_df = create_single_label_dataframe(outfile, image_contrast_map)
    
    full_label_df=full_label_df.append(single_label_df)

TCGA-CS-5396
TCGA-CS-6669
TCGA-HT-7480
TCGA-DU-5853
TCGA-CS-6186
TCGA-FG-A6J1
TCGA-HT-7684
TCGA-HT-8010
TCGA-DU-6404
TCGA-DU-5872
TCGA-HT-A4DV
TCGA-CS-6670
TCGA-DU-7015
TCGA-HT-7477
TCGA-FG-A4MT
TCGA-DU-8164
TCGA-DU-A5TR
TCGA-FG-7637
TCGA-HT-7604
TCGA-HT-8104
TCGA-HT-A61B
TCGA-FG-5963
TCGA-HT-8109
TCGA-HT-7603
TCGA-HT-7609
TCGA-HT-7690
TCGA-FG-5964
TCGA-HT-A5RA
TCGA-HT-7470
TCGA-DU-A5TU
TCGA-DU-8163
TCGA-FG-6688
TCGA-HT-7857
TCGA-DU-7018
TCGA-HT-7881
TCGA-HT-8110
TCGA-HT-7689
TCGA-HT-7610
TCGA-EZ-7265A
TCGA-DU-A6S7
TCGA-HT-7469
TCGA-HT-A617
TCGA-DU-5854
TCGA-FG-6691
TCGA-DU-7302
TCGA-DU-6410
TCGA-FG-8186
TCGA-HT-7902
TCGA-CS-4943
TCGA-DU-7306
TCGA-CS-5395
TCGA-DU-7298
TCGA-DU-A6S3
TCGA-HT-A5R7
TCGA-HT-8013
TCGA-HT-7687
TCGA-HT-8019
TCGA-HT-8114
TCGA-HT-A619
TCGA-HT-7467
TCGA-HT-7483
TCGA-DU-5871
TCGA-DU-6407
TCGA-HT-7694
TCGA-HT-8107
TCGA-HT-A61A
TCGA-HT-7607
TCGA-FG-7643
TCGA-FG-7634
TCGA-DU-6395
TCGA-DU-8167
TCGA-DU-5849
TCGA-HT-7474
TCGA-DU-6542
TCGA-HT-7854
TCGA-HT-7479
TCGA-HT-A5R

In [41]:
full_label_df.to_csv('/working/lupolab/julia/tcia_analysis/labels_tcia_lgg.csv', index = False)

In [5]:
full_label_df = pd.read_csv('/working/lupolab/julia/tcia_analysis/labels_tcia_lgg.csv')

In [6]:
full_label_df.head()

,ground_truth,image,sd
0,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,305-RT__COW-39894
1,T1,/working/lupolab/julia/tcia_analysis/datasets/...,1101-T1_AX_SE-54608
2,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,404-eB1000i-30080
3,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,301-3D_MOTSA_sense-80049
4,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,403-eBo-48146


In [7]:
full_label_df.loc[0]['image'].split('/')[7]

'TCGA-CS-5396'

In [8]:
full_label_df['patient_id'] = [x.split('/')[7] for x in full_label_df.image]

In [9]:
full_label_df['exam_id'] = [x.split('/')[8] for x in full_label_df.image]

In [10]:
full_label_df

,ground_truth,image,sd,patient_id,exam_id
0,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,305-RT__COW-39894,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
1,T1,/working/lupolab/julia/tcia_analysis/datasets/...,1101-T1_AX_SE-54608,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
2,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,404-eB1000i-30080,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
3,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,301-3D_MOTSA_sense-80049,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
4,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,403-eBo-48146,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
5,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,402-dADC_MAP-53538,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
6,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,304-LT_COW-25699,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
7,T1,/working/lupolab/julia/tcia_analysis/datasets/...,501-T1W__SE-58849,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
8,T2,/working/lupolab/julia/tcia_analysis/datasets/...,701-T2WTSE-70754,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605
9,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,901-FFE_FOR_BLEED-67362,TCGA-CS-5396,03-02-2001-MRI_BRAIN_CONTRAST_MRA_BRAIN-47605


In [11]:
patient_list = full_label_df.patient_id.unique()

In [12]:
len(patient_list)

199

In [13]:
usable_patients = []
for patient in patient_list: 
    patient_df = full_label_df.loc[(full_label_df['patient_id'] == patient) & (full_label_df['ground_truth'] != "OTHER")]
#     print(patient_df)
    if len(patient_df.ground_truth.unique()) >= 4: 
        usable_patients.append(patient)

        

In [14]:
len(usable_patients)

92

In [15]:
usable_patient_df = full_label_df.loc[(full_label_df['patient_id'].isin(usable_patients)) & 
                                      (full_label_df['ground_truth'] != "OTHER")].copy()

In [16]:
usable_patient_df

,ground_truth,image,sd,patient_id,exam_id
39,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,12-COR__T1_POST_GD_FLAIR-32689,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
41,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,11-AX_T1_POST_GD_FLAIR-81300,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
43,T1,/working/lupolab/julia/tcia_analysis/datasets/...,8-AX_T1_pre_gd-14019,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
44,T1,/working/lupolab/julia/tcia_analysis/datasets/...,9-3D_DCE_T1_MAP-73753,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
46,T2,/working/lupolab/julia/tcia_analysis/datasets/...,7-AX_T2_FR-FSE_RF2_150-55295,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
47,T2_FLAIR,/working/lupolab/julia/tcia_analysis/datasets/...,6-AXIAL_FLAIR-84159,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
48,T1,/working/lupolab/julia/tcia_analysis/datasets/...,3-SAG_T1_FLAIR-94337,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436
52,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,1001-T1_SE_POST-94212,TCGA-CS-6186,06-01-2000-MRI_BRAIN_per_R-97608
53,T2,/working/lupolab/julia/tcia_analysis/datasets/...,501-T2-WHOLE_BRAIN-32557,TCGA-CS-6186,06-01-2000-MRI_BRAIN_per_R-97608
54,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,1301-T1_SAG_POST-08017,TCGA-CS-6186,06-01-2000-MRI_BRAIN_per_R-97608


In [17]:
usable_patient_df['fixed_vol']  = ['' for x in usable_patient_df['ground_truth']]

In [18]:
for idx, row in usable_patient_df.iterrows():
    patient = row['patient_id']
    patient_df = usable_patient_df.loc[(full_label_df['patient_id'] == patient) & (full_label_df['ground_truth'] != "OTHER")]
    if 'T1' in patient_df['ground_truth'].unique(): 
        t1_df = patient_df[patient_df['ground_truth'] == "T1"]
        if t1_df.shape[0] > 1: 
            for idx, row1 in t1_df.iterrows(): 
                if 'ax' in row1['sd'].lower(): 
                    fixed_vol = row1['image']
                    break 
                else: 
                    fixed_vol = row1['image']
        else: 
            fixed_vol = list(t1_df['image'])[0]
    
    row['fixed_vol'] = fixed_vol
    

In [19]:
usable_patient_df.head()

,ground_truth,image,sd,patient_id,exam_id,fixed_vol
39,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,12-COR__T1_POST_GD_FLAIR-32689,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,/working/lupolab/julia/tcia_analysis/datasets/...
41,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,11-AX_T1_POST_GD_FLAIR-81300,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,/working/lupolab/julia/tcia_analysis/datasets/...
43,T1,/working/lupolab/julia/tcia_analysis/datasets/...,8-AX_T1_pre_gd-14019,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,/working/lupolab/julia/tcia_analysis/datasets/...
44,T1,/working/lupolab/julia/tcia_analysis/datasets/...,9-3D_DCE_T1_MAP-73753,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,/working/lupolab/julia/tcia_analysis/datasets/...
46,T2,/working/lupolab/julia/tcia_analysis/datasets/...,7-AX_T2_FR-FSE_RF2_150-55295,TCGA-DU-5853,08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,/working/lupolab/julia/tcia_analysis/datasets/...


In [20]:
usable_patient_df.to_csv('/working/lupolab/julia/tcia_analysis/code/usable_patient_lgg.csv')

Now that we finally have the fixed volume we want to align to for each image in each patient that we care about, let's go ahead and create the commands for brainsfit in brackets so that we can send them to the grid: 

`BRAINSFit --fixedVolume $fixed_vol.nii.gz --movingVolume $moving_vol.nii.gz --outputVolume $output_vol.nii.gz --outputTransform $transform_output.tfm  --transformType Rigid`

In [112]:
commands = []
for idx, row in usable_patient_df.iterrows(): 
    fixed_vol = row['fixed_vol']
    moving_vol = row['image']
    output_vol = moving_vol.split('.n')[0]+'a'+'.nii.gz'
    command_align = 'BRAINSFit --fixedVolume '+fixed_vol+' --movingVolume '+moving_vol+' --outputVolume '+output_vol+' --transformType Rigid'
    commands.append(command_align)
    

In [115]:
os.chdir('/working/lupolab/julia/tcia_analysis/code/')

In [118]:
# x = open('register_lggs.txt', 'w')

In [119]:
# x.write('{')
# x.write('\n')
# for command in commands: 
#     x.write(command)
#     x.write('\n')
# x.write('}')
# x.close()

In [120]:
!grid_pipeline --scriptonly register_lggs.txt
